# Convert model to onnx

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import warnings
warnings.simplefilter('ignore')

import gc

from os import path
import sys
sys.path.append(path.abspath('..'))

In [ ]:
import onnxruntime as ort
import onnx
import numpy as np
import torch
from src.lightning_module import PlanetsModule

In [ ]:
ONNX_MODEL_NAME = '../onnx_planet_model.onnx'
DEVICE = 'cpu'
BATCH_SIZE = 1

In [ ]:
# Загружаем модель
checkpoints_folder = '../experiments/experiment2/'
# Берем какой-то чекпоинт из папки. Если нужен конкретный - можно явно указать путь
checkpoint_name = os.path.join(checkpoints_folder, os.listdir(checkpoints_folder)[0]) 
module = PlanetsModule.load_from_checkpoint(checkpoint_name)

module.eval()
module.to(DEVICE)

In [ ]:
dummy_input = torch.rand(1, 3, 224, 224, device=DEVICE)
torch.onnx.export(
    module,
    dummy_input,
    ONNX_MODEL_NAME,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes = {'input': [0], 'output': [0]}, # динамический батч, но можно и статический
)

In [ ]:
# проверяем, что модель конвертнулась успешно
onnx_model = onnx.load(ONNX_MODEL_NAME)
onnx.checker.check_model(onnx_model)
print(onnx.helper.printable_graph(onnx_model.graph))